# 1. Data Sources & Collection

CMS Nursing Home Provider Information - https://data.cms.gov/provider-data/dataset/4pq5-n9py (2017 - 2021)

CMS Medicare Claims Quality Measures - https://data.cms.gov/provider-data/dataset/ijh5-nb2v (April 2020 - March 2021)

CMS Survey Summary - https://data.cms.gov/provider-data/dataset/tbry-pc2d (2017 - 2020)

CMS MDS Quality Measures - https://data.cms.gov/provider-data/dataset/djen-97ju (April 2020 - March 2021)

CMS COVID-19 Nursing Home Data - https://data.cms.gov/covid-19/covid-19-nursing-home-data (April 2020 - December 2020, January 2021 - March 2021)

# 2. Construct dataset

In [2]:
import xgboost
import pandas as pd 
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats

In [3]:
# Set directory
os.chdir('C:/Users/erpin/Documents/CIND820')

In [4]:
p = 'NH_ProviderInfo_Jan2022.csv'
s = 'NH_SurveySummary_Nov2021.csv'
m = 'NH_QualityMsr_MDS_Nov2021.csv'
c = 'NH_QualityMsr_Claims_Nov2021.csv'
cov = 'faclevel_2020.csv'

# Select columns from provider dataset
dfp = pd.read_csv(p, dtype={'Federal Provider Number':'str'}, encoding='cp1252')
colsp= [0,1,4,9,10,11,30,48,50,58,59,60,87,88,89]
dfp = dfp[dfp.columns[colsp]]
dfp['Federal Provider Number'] = dfp['Federal Provider Number'].apply(lambda x: x.lstrip('0'))

# Select columns from MDS dataset 
dfm = pd.read_csv(m, dtype={'Federal Provider Number':'str'}, encoding='cp1252')
LS = dfm[dfm['Resident type']== 'Long Stay']
colsm = [0,7,8,17]
LS = LS[LS.columns[colsm]]
LSpivot = LS.pivot(index=['Federal Provider Number', 'Resident type'], columns='Measure Description', values='Four Quarter Average Score')
LSpivot.reset_index(drop=False, inplace=True)
LSpivot['Federal Provider Number'] = LSpivot['Federal Provider Number'].apply(lambda x: x.lstrip('0'))

# Select columns from surveys dataset 
dfs = pd.read_csv(s, dtype={'Federal Provider Number':'str'}, encoding='cp1252')
colss = [0,6,7,9,10]
dfs = dfs[dfs.columns[colss]]
dfs = dfs[dfs['Inspection Cycle']==3]
dfs.to_csv('inspection.csv')
dfs['Federal Provider Number'] = dfs['Federal Provider Number'].apply(lambda x: x.lstrip('0'))

#select columns from COVID 2020 dataset 
dfcov = pd.read_csv(cov, dtype={'Federal Provider Number':'str'})
colcov = [1,13,19,22]
dfcov = dfcov[dfcov.columns[colcov]]
dfcov['Total Number of Occupied Beds'] = dfcov['Total Number of Occupied Beds'].replace(0, np.nan)
dfcov['Confirmed COVID-19 Cases Per Occupied Beds'] = dfcov['Residents Weekly Confirmed COVID-19']/dfcov['Total Number of Occupied Beds']
dfcov['COVID-19 Deaths Per Occupied Beds'] = dfcov['Residents Weekly COVID-19 Deaths']/dfcov['Total Number of Occupied Beds']
dfcovm = dfcov.groupby(['Federal Provider Number']).mean().reset_index()
dfcovm['Federal Provider Number'] = dfcovm['Federal Provider Number'].apply(lambda x: x.lstrip('0'))

#Select columns from claims dataset 
dfc = pd.read_csv(c, dtype={'Federal Provider Number':'str'}, encoding='cp1252')
dfc = dfc[dfc['Measure Description']=='Number of outpatient emergency department visits per 1000 long-stay resident days']
colc = [0,7,9]
dfc = dfc[dfc.columns[colc]]
dfc['Federal Provider Number'] = dfc['Federal Provider Number'].apply(lambda x: x.lstrip('0'))

df1 = pd.merge(dfp, LSpivot, on='Federal Provider Number', how='left')
df2 = pd.merge(df1, dfs, on='Federal Provider Number', how='left')
df3 = pd.merge(df2, dfcovm, on='Federal Provider Number', how='left')
final_df = pd.merge(df3, dfc, on='Federal Provider Number', how='left')

C:\Users\erpin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (84,85) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))